# This workbook was made for use in UCCSU Student Council voting

In [19]:
# Importing necessary modules
import pandas as pd
import tabulate as tb
import math

### Importing the ballots and cleaning up the file

In [20]:
# Imports a data file named "Ballots.csv"
data = pd.read_csv("Ballots.csv")

# Removing timestamp
data.drop('Timestamp', inplace=True, axis=1)

### Eliminating any votes that are not in the list of Council Members

In [21]:
# Imports the list of emails
emails = pd.read_csv("Emails.csv")

# Checks every entry in the "Email Address" column of the data file and adds it to the ballots file if there is an entry in the "Members" column of the emails file
ballots = data[data["Email Address"].isin(emails["Members"])].fillna(0)

### Seperating the combined ballots into ballots for each position

In [22]:
# President
pres_ballots = ballots[[
    "President [Clare Austick]", 
    "President [RON]"
]].astype('Int64')

pres_ballots.columns = [
    "Clare Austick",
    "RON"
]

# VP for Academic Affairs
academic_ballots = ballots[[
    "VP for Academic Affairs [Eimear Curtin]", 
    "VP for Academic Affairs [Megan O'Connor]", 
    "VP for Academic Affairs [RON]"
]].astype('Int64')

academic_ballots.columns = [
    "Eimear Curtin",
    "Megan O'Connor",
    "RON"
]

# VP for Welfare
welfare_ballots = ballots[[
    "VP for Welfare [Somhairle Brennan]",
    "VP for Welfare [RON]"
]].astype('Int64')

welfare_ballots.columns = [
    "Somhairle Brennan",
    "RON"
]

# VP for Campaigns
campaigns_ballots = ballots[[
    "VP for Campaigns [Beth O'Reilly]",
    "VP for Welfare [RON]"
]]

campaigns_ballots.columns = [
    "Beth O'Reilly",
    "RON"
]

# VP for Equality & Citizenship
equality_ballots = ballots[[
    "VP for Equality & Citizenship [Bukky Adebowale]",
    "VP for Equality & Citizenship [Luke Daly]",
    "VP for Equality & Citizenship [RON]"
]]

equality_ballots.columns = [
    "Bukky Adebowale",
    "Luke Daly",
    "RON"
]

# Leas-Uachtarán don Ghaeilge
gaeilge_ballots = ballots[[
    "Leas-Uachtarán don Ghaeilge [Muireann Nic Corcráin]",
    "Leas-Uachtarán don Ghaeilge [Grian Ní Dhaimhin]",
    "Leas-Uachtarán don Ghaeilge [RON]"
]]

gaeilge_ballots.columns = [
    "Muireann Nic Corcráin",
    "Grian Ní Dhaimhin",
    "RON"
]

# VP for Postgraduate-Affairs
postgrad_ballots = ballots[[
    "VP for Postgraduate Affairs [Jenna Barry]",
    "VP for Postgraduate Affairs [RON]"
]]

postgrad_ballots.columns = [
    "Jenna Barry",
    "RON"
]

# VP for the Southern Region
southern_ballots = ballots[[
    "VP for the Southern Region [John Fortune]",
    "VP for the Southern Region [Nutatwud Nutchanat]",
    "VP for the Southern Region [RON]"
]]

southern_ballots.columns = [
    "John Fortune",
    "Nutatwud Nutchanat",
    "RON"
]

### Defining method for counting the votes

#### Counts the number of preferences each candidate received

In [23]:
def tally(ballots, candidate):
    
    # Initialising preference counts
    pref_1 = 0
    pref_2 = 0
    pref_3 = 0
    
    # Counting number of preferences
    for j in ballots[candidate]:
        if j == 1:
            pref_1 += 1
        if j == 2:
            pref_2 += 1
        if j == 3:
            pref_3 += 1
    
    return pref_1, pref_2, pref_3

### Gets the candidate(s) with the lowest value pref

In [24]:
def get_indexes_min_value(list):
    
    # Removing 0's from the list
    for i in list:
        if i == 0:
            list.pop(i)
    # What is the minimum value
    min_value = min(list)
    
    # If there are more than one minimum value (tied last) then return both indices
    if list.count(min_value) > 1:
        return [i for i, x in enumerate(list) if x == min(list)]
    
    # Else return the index of the minimum value
    else:
        return list.index(min(list))

### Gets the candidate(s) with the highest value pref

In [25]:
def get_indexes_max_value(list):
    
    # Wat is the maximum value
    max_value = max(list)
    
    # If there are more than one maximum value then return both indices
    if list.count(max_value) > 1:
        return [i for i, x in enumerate(list) if x == max(list)]
    
    # Else return the index of the maximum value
    else:
        return list.index(max(list))

### Calculates the valid poll and droop quota

In [26]:
def quota(ballots):
    
    candidates = ballots.columns.values.tolist()
    votes_with_no_first_preference = ballots.copy()
    
    # Calculating how many votes have no preferences
    for i in candidates:
        votes_with_no_first_preference = votes_with_no_first_preference[votes_with_no_first_preference[i] != 1]
    total_invalid_votes = len(votes_with_no_first_preference)
        
    # Calculate the valid poll and droop quota
    valid_poll = len(ballots) - total_invalid_votes
    droop_quota = math.floor(valid_poll / 2) + 1
    
    return droop_quota, valid_poll

### Prints the tallies as they stand

In [27]:
def print_results(ballots):
    
    # Gets the list of Candidates
    candidates = ballots.columns.values.tolist()
    # Initialises the output table with the headings
    output = [["Candidate", "1st Preference", "2nd Preference", "3rd Preference"]]
    
    #Calculates the total pref for each candidate and appends it to output table
    for i in candidates:
        pref_1, pref_2, pref_3 = tally(ballots, i)
        output.append([i, str(pref_1), str(pref_2), str(pref_3)])
        
    print(tb.tabulate(output, headers="firstrow") + "\n")

### Checks if a candidate has reached a quota and eliminates the candidate with the lowest 1st preference if not

In [60]:
def eliminate(ballots):
    
    candidates = ballots.columns.values.tolist()
    count = []
    
    for i in candidates:
        pref_1, pref_2, pref_3 = tally(ballots, i)        
        count.append([i, pref_1, pref_2, pref_3])
    
    # Checks the first preferences
    all_pref_1 = [row[1] for row in count]
    
    # Check if any candidate has reached the quota
    highest = max(all_pref_1)
    droop_quota, valid_poll = quota(ballots)
    
    if highest >= droop_quota:
        highest_candidate = get_indexes_max_value(all_pref_1)
        print(candidates[highest_candidate] + " has exceeded the quota and is deemed elected \n")
        return True
    
    # Get the lowest candidate(s) if no winner
    eliminated = get_indexes_min_value(all_pref_1)
    
    # If only one candidate has the lowest 1st preferences then eliminate them and redistribute preferences
    if type(eliminated) == int:
        print(candidates[eliminated] + " is eliminated as they have the lowest 1st preferences of remaining candidates \n")
        eliminated_ballots = redistribute_preferences(ballots, candidates[eliminated])
    
    # If multiple candidates have tied lowest 1st preferences then tally their second and eliminate
    elif len(eliminated) > 1:
        all_pref_2 = [row[2] for row in count]
        eliminated_2 = get_indexes_min_value(all_pref_2)
        
        if type(eliminated_2) == int:
            print(candidates[eliminated[0]] + " & " + candidates[eliminated[1]] + " had a tied 1st preference so " + candidates[eliminated_2] + " was eliminated on 2nd preferences \n")
            eliminated_ballots = redistribute_preferences(ballots, candidates[eliminated_2])
        
        # If still tied then print below as coin flip will have to be done
        else:
            print("Multiple candidates have tied preferences \n")
            return False
    
    return eliminated_ballots

### Redistributes the preferences of the eliminated candidate

In [45]:
def redistribute_preferences(ballots, candidate):
    
    # Getting the index of each vote that has the eliminated candidate as 1st preference
    votes_to_distribute_index = ballots.loc[ballots[candidate] == 1].index.tolist()
    
    # Dataframe with all votes that have preferences that need distributing
    votes_to_distribute = ballots.loc[votes_to_distribute_index, :]
    
    # Replacing 2nd Pref with 1st Pref etc.
    for i in range(len(votes_to_distribute)):
        for j in range(len(ballots.columns.values.tolist())):
            for k in range(1, len(ballots.columns.values.tolist()) + 1):
                if votes_to_distribute.iloc[i, j] == k:
                    votes_to_distribute.iloc[i, j] = k-1
    
    # Merging this back to the full ballots
    updated_ballots = ballots.copy()
    updated_ballots.update(votes_to_distribute)
    
    return updated_ballots

### Main function combining the above methods

In [46]:
def main(ballots):
    
    active_ballots = ballots
    
    for i in range(0, len(ballots.columns) - 1):
        
        # Print Count
        print("Count " + str(i+1))
        
        # Print Quota
        droop_quota, valid_poll = quota(active_ballots)
        print("Total valid poll is = " + str(valid_poll))
        print("Quota is = " + str(droop_quota) + "\n")
        
        # Print Results
        print_results(active_ballots)
        
        #Check for winner and eliminate if none
        active_ballots = eliminate(active_ballots)
        
        # True is if winner is found. False is if a winner couldn't be determined
        if type(active_ballots) == bool:
            if active_ballots == True:
                print("Winner has been found")
            if active_ballots == False:
                print("No winner could be determined")

# The Count

In [47]:
main(pres_ballots)

Count 1
Total valid poll is = 31
Quota is = 16

Candidate        1st Preference    2nd Preference    3rd Preference
-------------  ----------------  ----------------  ----------------
Clare Austick                14                13                 0
RON                          17                 9                 0

RON has exceeded the quota and is deemed elected 

Winner has been found


In [48]:
main(academic_ballots)

Count 1
Total valid poll is = 30
Quota is = 16

Candidate         1st Preference    2nd Preference    3rd Preference
--------------  ----------------  ----------------  ----------------
Eimear Curtin                 10                 7                 7
Megan O'Connor                 8                13                 5
RON                           12                 4                 9

Megan O'Connor is eliminated as they have the lowest 1st preferences of remaining candidates 

Count 2
Total valid poll is = 28
Quota is = 15

Candidate         1st Preference    2nd Preference    3rd Preference
--------------  ----------------  ----------------  ----------------
Eimear Curtin                 13                 6                 5
Megan O'Connor                 0                13                 5
RON                           15                 3                 7

RON has exceeded the quota and is deemed elected 

Winner has been found


In [49]:
main(welfare_ballots)

Count 1
Total valid poll is = 29
Quota is = 15

Candidate            1st Preference    2nd Preference    3rd Preference
-----------------  ----------------  ----------------  ----------------
Somhairle Brennan                15                 9                 0
RON                              14                12                 0

Somhairle Brennan has exceeded the quota and is deemed elected 

Winner has been found


In [50]:
main(campaigns_ballots)

Count 1
Total valid poll is = 27
Quota is = 14

Candidate        1st Preference    2nd Preference    3rd Preference
-------------  ----------------  ----------------  ----------------
Beth O'Reilly                18                11                 0
RON                          14                12                 0

Beth O'Reilly has exceeded the quota and is deemed elected 

Winner has been found


In [51]:
main(equality_ballots)

Count 1
Total valid poll is = 30
Quota is = 16

Candidate          1st Preference    2nd Preference    3rd Preference
---------------  ----------------  ----------------  ----------------
Bukky Adebowale                10                11                 5
Luke Daly                       7                13                 5
RON                            13                 1                10

Luke Daly is eliminated as they have the lowest 1st preferences of remaining candidates 

Count 2
Total valid poll is = 30
Quota is = 16

Candidate          1st Preference    2nd Preference    3rd Preference
---------------  ----------------  ----------------  ----------------
Bukky Adebowale                17                 4                 5
Luke Daly                       0                13                 5
RON                            13                 5                 6

Bukky Adebowale has exceeded the quota and is deemed elected 

Winner has been found


In [52]:
main(gaeilge_ballots)

Count 1
Total valid poll is = 31
Quota is = 16

Candidate                1st Preference    2nd Preference    3rd Preference
---------------------  ----------------  ----------------  ----------------
Muireann Nic Corcráin                 9                 8                 8
Grian Ní Dhaimhin                     8                13                 5
RON                                  14                 6                 7

Grian Ní Dhaimhin is eliminated as they have the lowest 1st preferences of remaining candidates 

Count 2
Total valid poll is = 31
Quota is = 16

Candidate                1st Preference    2nd Preference    3rd Preference
---------------------  ----------------  ----------------  ----------------
Muireann Nic Corcráin                14                 4                 7
Grian Ní Dhaimhin                     0                13                 5
RON                                  17                 5                 5

RON has exceeded the quota and is deemed ele

In [61]:
main(postgrad_ballots)

Count 1
Total valid poll is = 30
Quota is = 16

Candidate      1st Preference    2nd Preference    3rd Preference
-----------  ----------------  ----------------  ----------------
Jenna Barry                15                11                 0
RON                        15                10                 0

Jenna Barry & RON had a tied 1st preference so RON was eliminated on 2nd preferences 



In [54]:
main(southern_ballots)

Count 1
Total valid poll is = 30
Quota is = 16

Candidate             1st Preference    2nd Preference    3rd Preference
------------------  ----------------  ----------------  ----------------
John Fortune                      14                 5                 7
Nutatwud Nutchanat                 6                16                 4
RON                               10                 5                12

Nutatwud Nutchanat is eliminated as they have the lowest 1st preferences of remaining candidates 

Count 2
Total valid poll is = 30
Quota is = 16

Candidate             1st Preference    2nd Preference    3rd Preference
------------------  ----------------  ----------------  ----------------
John Fortune                      19                 0                 7
Nutatwud Nutchanat                 0                16                 4
RON                               11                 7                 9

John Fortune has exceeded the quota and is deemed elected 

Winner has be